### 네이버 브랜드스토어 데이터 수집
- 네이버 브랜드스토어의 상품 데이터나 리뷰 데이터등을 수집 및 분석
- 조별로 관심있는 분야의 상품 데이터를 수집후 자율적으로 분석수행
- 여러개의 상품을 수집해서 비교 분석해도 되고, 특정 상품을 수집해서 아래의 예시와 같이 분석해도 됩니다.

#### 예시
- 상하목장 네이버 스토어 페이지 리뷰 데이터 수집
    - 매일유업 스토어 페이지 : https://brand.naver.com/maeil
    - 데이터 수집 상품 : 유기농[상하목장]
    - 상품 페이지 : https://brand.naver.com/maeil/products/4568404023

In [1]:
import time
import requests
import numpy as np
import pandas as pd

In [2]:
# 1. 데이터 수집

In [2]:
page, page_size = 1, 30
url = f"https://brand.naver.com/n/v1/reviews/paged-reviews?page={page}&pageSize={page_size}&merchantNo=510133398&originProductNo=4558447772&sortType=REVIEW_CREATE_DATE_DESC"
response = requests.get(url)
response

<Response [200]>

In [33]:
# 2. 데이터 전처리

In [3]:
columns = ["id", "reviewContent", "createDate", "reviewScore"]
datas = response.json()["contents"]
df = pd.DataFrame(datas)[columns]
df["createTime"] = df["createDate"].apply(lambda data:data[11:19])
df["createDate"] = df["createDate"].apply(lambda data:data[:10])
df.tail(2)

,id,reviewContent,createDate,reviewScore,createTime
28,3373901827,딸아이가 너무 좋아하는 우유예요~^^,2022-02-23,5,06:10:41
29,3373887201,이가 매일 3개씩 먹고있어요,2022-02-23,5,05:52:55


In [35]:
# 3. 함수로 만들기

In [4]:
def sanghafarm(page=1, page_size=30):
    url = f"https://brand.naver.com/n/v1/reviews/paged-reviews?page={page}&pageSize={page_size}&merchantNo=510133398&originProductNo=4558447772&sortType=REVIEW_CREATE_DATE_DESC"
    response = requests.get(url)
    columns = ["id", "reviewContent", "createDate", "reviewScore"]
    datas = response.json()["contents"]
    df = pd.DataFrame(datas)[columns]
    df["createTime"] = df["createDate"].apply(lambda data:data[11:19])
    df["createDate"] = df["createDate"].apply(lambda data:data[:10])
    return df

In [13]:
# 4. 반복문으로 데이터 수집 : 최근 데이터 600개

In [5]:
%%time
dfs = []
for page in range(1, 20):
    print(page, end=" ")
    dfs.append(sanghafarm(page=1, page_size=30))
    time.sleep(np.random.randint(1, 4))
    
review_df = pd.concat(dfs, ignore_index=True)
review_df.tail(2)
review_df.to_csv("datas/reviews.csv", index=False)

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 Wall time: 45.8 s


In [7]:
# 상품페이지에 있는 리뷰 데이터 수집
review_df = pd.read_csv("datas/reviews.csv")
review_df.tail()

,id,reviewContent,createDate,reviewScore,createTime
565,3373980697,상하목장 유기농 담백하고 부담없어요,2022-02-23,3,07:42:36
566,3373953021,원래 먹던우유예요 ㅡ아들이 시키다가 이번에ㅡ다시시켔는데즈좋아요,2022-02-23,5,07:11:43
567,3373915963,잘 받았습니다 빠른 배송 감사합니다,2022-02-23,5,06:27:41
568,3373901827,딸아이가 너무 좋아하는 우유예요~^^,2022-02-23,5,06:10:41
569,3373887201,이가 매일 3개씩 먹고있어요,2022-02-23,5,05:52:55


In [8]:
import requests, re
from bs4 import BeautifulSoup

product_no = 2540929238
url = f"https://brand.naver.com/maeil/products/{product_no}"
response = requests.get(url)
origin_product_no = re.findall('DiscountPolicyNo":"SE_([0-9]+)",', response.text)[0]
origin_product_no

'2538027516'